# Atividade 01

### Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
#Importando as bibliotecas
import pandas as pd
import requests

In [2]:
# Lê o arquivo CSV e carrega os dados em um DataFrame
df = pd.read_csv('SINASC_RO_2019.csv')

# Imprime o número total de registros no DataFrame
print(f'Número total de registros: {len(df)}')

# Verifica se há linhas duplicadas e imprime o resultado
print(f'Há linhas duplicadas: {len(df) != len(df.drop_duplicates())}')

Número total de registros: 27028
Há linhas duplicadas: False


In [3]:
# Imprime a contagem de valores ausentes por variável no DataFrame
print(f'Valores missing por variável:\n{df.isnull().sum()}')

Valores missing por variável:
ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [4]:
# Lê o arquivo CSV e seleciona apenas as colunas de interesse
df_interesse = pd.read_csv('SINASC_RO_2019.csv')[[
    'LOCNASC', 
    'IDADEMAE', 
    'ESTCIVMAE', 
    'ESCMAE', 
    'QTDFILVIVO', 
    'GESTACAO', 
    'GRAVIDEZ', 
    'CONSULTAS', 
    'APGAR5']]

# Imprime o DataFrame resultante com as colunas de interesse
print(f'DataFrame com colunas de interesse:\n{df_interesse}')

DataFrame com colunas de interesse:
       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5  
0      37 a 41 semanas    Única          4    10.0  
1      37 a 41 semanas    Única          

In [5]:
# Imprime a contagem de valores ausentes nas colunas de interesse no DataFrame
print(f'Valores missing nas colunas de interesse:\n{df_interesse.isnull().sum()}')

Valores missing nas colunas de interesse:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [6]:
# Remove os registros em que a coluna 'APGAR5' possui valores ausentes
df_sem_apgar5_missing = df_interesse.dropna(subset=['APGAR5'])

# Imprime o DataFrame resultante após a remoção
print(f'\nDataFrame após remover registros com APGAR5 não preenchido:\n{df_sem_apgar5_missing}')

# Calcula e imprime o número de registros após a remoção
num_registros_sem_apgar5_missing = len(df_sem_apgar5_missing)
print(f'Número de registros após remoção: {num_registros_sem_apgar5_missing}')

# Calcula e imprime os valores ausentes nas colunas de interesse após a remoção
missings_apgar5 = df_sem_apgar5_missing.isnull().sum()
print(f'Valores missing nas colunas de interesse após remoção:\n{missings_apgar5}')


DataFrame após remover registros com APGAR5 não preenchido:
       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5  
0      37 a 41 semanas    Única          4    10.0  
1      37 a 41 s

In [7]:
# Lê o arquivo CSV e seleciona apenas as colunas de interesse
df_interesse = pd.read_csv('SINASC_RO_2019.csv')[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]

# Imprime o DataFrame resultante com as colunas de interesse e a contagem de valores ausentes nas colunas
print(f'DataFrame com colunas de interesse:\n{df_interesse}\nValores missing nas colunas de interesse:\n{df_interesse.isnull().sum()}')

# Substitui os códigos 9 por NaN em ESTCIVMAE e CONSULTAS
df_interesse.replace({'ESTCIVMAE': 9, 'CONSULTAS': 9}, pd.NA, inplace=True)

# Imprime o DataFrame após a substituição
print(f'\nDataFrame após substituir código 9 por NaN em ESTCIVMAE e CONSULTAS:\n{df_interesse}')

DataFrame com colunas de interesse:
       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5  
0      37 a 41 semanas    Única          4    10.0  
1      37 a 41 semanas    Única          

In [8]:
# Substitui valores faltantes em 'QTDFILVIVO' por zero no DataFrame
df_sem_apgar5_missing['QTDFILVIVO'].fillna(0, inplace=True)

# Imprime o DataFrame resultante após a substituição
print(f'\nDataFrame após substituir valores faltantes em QTDFILVIVO por zero:\n{df_sem_apgar5_missing}')


DataFrame após substituir valores faltantes em QTDFILVIVO por zero:
       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5  
0      37 a 41 semanas    Única          4    10.0  
1      3

In [9]:
# Substitui valores faltantes em 'IDADEMAE' pela mediana no DataFrame
df_sem_apgar5_missing['IDADEMAE'].fillna(df_sem_apgar5_missing['IDADEMAE'].median(), inplace=True)
print(f'\nDataFrame após substituir valores faltantes em IDADEMAE pela mediana:\n{df_sem_apgar5_missing.head()}')

# Substitui valores faltantes em 'ESCMAE' pela categoria mais comum
df_sem_apgar5_missing['ESCMAE'].fillna(df_sem_apgar5_missing['ESCMAE'].mode()[0], inplace=True)
print(f'\nDataFrame após substituir valores faltantes em ESCMAE pela categoria mais comum:\n{df_sem_apgar5_missing.head()}')

# Substitui valores faltantes em variáveis categóricas ('GESTACAO', 'GRAVIDEZ', 'LOCNASC') pelos modos
df_sem_apgar5_missing[['GESTACAO', 'GRAVIDEZ', 'LOCNASC']] = df_sem_apgar5_missing[['GESTACAO', 'GRAVIDEZ', 'LOCNASC']].fillna(df_sem_apgar5_missing.mode().iloc[0])
print(f'\nDataFrame após substituir valores faltantes em variáveis categóricas:\n{df_sem_apgar5_missing.head()}')

# Substitui valores faltantes em 'CONSULTAS' pela mediana
df_sem_apgar5_missing['CONSULTAS'].fillna(df_sem_apgar5_missing['CONSULTAS'].median(), inplace=True)
print(f'\nDataFrame após substituir valores faltantes em CONSULTAS pela mediana:\n{df_sem_apgar5_missing.head()}')


DataFrame após substituir valores faltantes em IDADEMAE pela mediana:
   LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO         GESTACAO  \
0        1        19        5.0      8 a 11 anos         0.0  37 a 41 semanas   
1        1        29        2.0      8 a 11 anos         1.0  37 a 41 semanas   
2        1        37        9.0      8 a 11 anos         2.0  37 a 41 semanas   
3        1        30        5.0  12 anos ou mais         0.0  37 a 41 semanas   
4        1        30        2.0      8 a 11 anos         1.0  37 a 41 semanas   

  GRAVIDEZ  CONSULTAS  APGAR5  
0    Única          4    10.0  
1    Única          4     9.0  
2    Única          4    10.0  
3    Única          3    10.0  
4    Única          4    10.0  

DataFrame após substituir valores faltantes em ESCMAE pela categoria mais comum:
   LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO         GESTACAO  \
0        1        19        5.0      8 a 11 anos         0.0  37 a 41 semanas   
1     

### Justificativa das tratativas:

IDADEMAE: Optei por preencher com a mediana, que é o valor que está no meio dos dados, para manter as idades próximas do que já tínhamos.

ESCMAE: Preenchi com a categoria que aparecia mais vezes, a mais comum, para não mudar muito a distribuição das informações.

GESTACAO, GRAVIDEZ, LOCNASC: Coloquei os valores mais comuns, acreditando que representam bem a maioria dos casos.

CONSULTAS: Optei por usar a mediana para não ser muito influenciado por valores muito altos ou baixos.

Busquei equilibrar entre manter as características originais dos dados e inserir informações que fazem sentido

In [10]:
# Cria uma nova coluna 'APGAR_CATEGORIA' com base na categorização de 'APGAR5'
df_sem_apgar5_missing['APGAR_CATEGORIA'] = pd.cut(df_sem_apgar5_missing['APGAR5'], bins=[-1, 3, 5, 7, 10], labels=['Severe Asphyxia', 'Moderate Asphyxia', 'Mild Asphyxia', 'Normal'])

# Imprime o DataFrame após a categorização
print(f'\nDataFrame após categorizar Apgar:\n{df_sem_apgar5_missing}')

# Calcula e imprime as frequências da categorização de 'APGAR5'
frequencias_apgar_categoria = df_sem_apgar5_missing['APGAR_CATEGORIA'].value_counts()
print(f'Frequências da categorização de Apgar:\n{frequencias_apgar_categoria}')


DataFrame após categorizar Apgar:
       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5 APGAR_CATEGORIA  
0      37 a 41 semanas    Única          4    10.0          Normal  
1      37 

In [11]:
# Renomeia as variáveis para snake case no DataFrame
df_sem_apgar5_missing.columns = df_sem_apgar5_missing.columns.str.lower().str.replace(' ', '_')

# Imprime o DataFrame após o processo de renomeação
print(f'\nDataFrame após renomear variáveis para snake case:\n{df_sem_apgar5_missing}')


DataFrame após renomear variáveis para snake case:
       locnasc  idademae  estcivmae           escmae  qtdfilvivo  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              gestacao gravidez  consultas  apgar5 apgar_categoria  
0      37 a 41 semanas    Única          4    10.0          No

# Atividade 02

O nosso projeto desta sequência de módulos do curso será um aprofundamento da demonstração sobre classificação de risco de crédito que vimos lá no comecinho. Pois recebemos uma base já montada pra nós. Tenha certeza de que ela passou por um longo processamento até ficar daquele jeito. Neste exercício vamos exercitar o que aprendemos nas ultimas aulas e montar a variável resposta da base do nosso projeto.

#### Marcação de bom e mau
O objetivo da modelagem é classificar o risco de inadimplência, ou como se diz no meio, o risco de *default*. Podemos fazer longas discussões sobre o conceito de *default* com base em estudos e exigências regulatórias, para efeitos deste estudo, um cliente em *default* é aquele que está em 60 dias de atraso ou mais. Então classificaremos os clientes como 'bons' e 'maus' assim:
- **Maus** pagadores: são aqueles que entraram em 'default' (atraso 60 dias ou mais) nos 24 meses seguintes à aquisição do cartão de crédito. 
- **Bons** pagadores: são considerados todos os demais.
- **Excluídos**: Clientes que não adquiriram um cartão de crédito (seja por recusa, seja por desistência) não possuem informações de pagamento, portanto não se pode identificar se são bons ou maus. Há uma longa discussão e literatura sobre *inferência de rejeitados* que está fora do escopo deste exercício.

#### Bases disponíveis
Temos duas bases importantes aqui: uma de propostas, com diversas informações dos vários solicitantes de cartão de crédito, e uma base de pagamentos. A base de pagamentos será utilizada para identificar a ocorrência de *default*. A base de propostas tem diversas informações coletadas no momento da solicitação do crédito (isto é importante: qualquer informação posterior a essa data é impossível de ser coletada na aplicação do modelo e não pode ser utilizada).

As variáveis delas são:

Base de propostas - application_records.csv

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| CODE_GENDER| M = 'Masculino'; F = 'Feminino' |M/F|
| FLAG_OWN_CAR| Y = 'possui'; N = 'não possui' |Y/N|
| FLAG_OWN_REALTY| Y = 'possui'; N = 'não possui' |Y/N|
| CNT_CHILDREN| Quantidade de filhos |inteiro|
| AMT_INCOME_TOTAL| Annual income |inteiro|
| NAME_INCOME_TYPE|Tipo de renda (ex: assaliariado, autônomo etc) | texto |
| NAME_EDUCATION_TYPE| Nível de educação (ex: secundário, superior etc) |texto|
| NAME_FAMILY_STATUS | Estado civil (ex: solteiro, casado etc)| texto |
| NAME_HOUSING_TYPE | tipo de residência (ex: casa/apartamento, com os pais etc) | texto |
| DAYS_BIRTH | Count backwards from current day (0), -1 means yesterday |inteiro|
| DAYS_EMPLOYED | Count backwards from current day (0), -1 means yesterday |inteiro|
| FLAG_MOBIL | Indica se possui celular (1 = sim, 0 = não) |binária|
| FLAG_WORK_PHONE | Indica se possui telefone comercial (1 = sim, 0 = não) |binária|
| FLAG_PHONE | Indica se possui telefone (1 = sim, 0 = não) |binária|
| FLAG_EMAIL | Indica se possui e-mail (1 = sim, 0 = não) |binária|
| OCCUPATION_TYPE | Occupation	 |Qualitativa|
| CNT_FAM_MEMBERS | quantidade de pessoas na residência |inteiro|

Base de pagamentos - pagamentos_largo.csv  

| Nome da Variável         | Description                                         | Tipo  |
| ------------------------ |:---------------------------------------------------:| -----:|
| ID| identificador do cliente (chave) |inteiro|
| mes_00 a mes_24| faixa de atraso mês a mês do cliente <br>0: 1-29 days past due &nbsp;&nbsp;&nbsp;&nbsp; 1: 30-59 days past due <br />2: 60-89 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 3: 90-119 days overdue <br /> 4: 120-149 days overdue &nbsp;&nbsp;&nbsp;&nbsp; 5: more than 150 days <br />C: paid off that month &nbsp;&nbsp;&nbsp;&nbsp; X: No loan for the month |Qualitativa|

#### Construindo a variável resposta
A base de pagamentos está em um formato de 'base larga'. Essa base possui informações de pagamentos do cliente mês a mês a partir do mês de aquisição do crédito (mês 0) até o vigésimo quarto mês após a aquisição do crédito (mês 24). Utilizaremos essa base para determinar se um proponente é considerado 'bom pagador' ou caso apresente atraso representativo, será considerado 'mau pagador'.

#### Base larga vs base longa
A base ser larga significa que há uma linha para cada cliente, e que as informações estarão nas colunas, em contraste com a 'base longa', em que haveria uma linha para cada combinação cliente/mês, uma coluna indicando o cliente, outra indicando o mês, e apenas uma coluna com a informação do atraso.

#### Tarefa 1) Marcar *default* no mês
Faça uma indicadora de se o cliente está em *default* em cada uma das marcações (mes_00 a mes_24). Dica: você pode utilizar o método ```.isin()``` do Pandas. Consulte a [documentação](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) caso necessário.

#### Tarefa 2) 'bons' e 'maus' ao longo de todos os 24 meses de desempenho
Marque para cada cliente se ele teve pelo menos um episódio de *default* entre o mês 0 e o mês 24. Dica: o método ```sum()``` pode ajudar. Caso precise, consulte a [documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html) e procure pelo argumento ```axis```, você viu outros métodos que possuem esse argumento também. Tendo o número de meses em default de cada cliente, basta marcar ```True``` para todos aqueles que possuem pelo menos 1 mês em *default* e ```False``` para os demais.

#### Tarefa 3) Marcando proponentes expostos ao risco de crédito
Marcando proponentes que se tornaram tomadores: lembre-se de que clientes que não adquiriram o cartão devem ser desconsiderados. A base de pagamentos possui apenas clientes que adquiriram cartão de crédito, então você pode selecionar somente os clientes da base de propostas que se encontram na base de pagamentos.

#### Tarefa 4) Consolidando as informações
Faça uma junção das informações da base de propostas com a variável de *default* que você acabou de construir. Talvez você consiga realizar a tarefa 3 e tarefa 4 em uma única linha de código ;)

#### Tarefa 5) Verificando
Faça uma contagem dos valores do *default* que você construiu. 

In [12]:
# Lê o arquivo CSV 'application_record.csv' e carrega os dados em um DataFrame chamado 'propostas'
propostas = pd.read_csv('application_record.csv')

# Exibe as primeiras linhas do DataFrame 'propostas'
propostas.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [13]:
# Lê o arquivo CSV 'pagamentos_largo.csv' e carrega os dados em um DataFrame chamado 'pagamento'
pagamento = pd.read_csv('pagamentos_largo.csv')

# Exibe as primeiras linhas do DataFrame 'pagamento'
pagamento.head()

,ID,mes_0,mes_1,mes_10,mes_11,mes_12,mes_13,mes_14,mes_15,mes_16,...,mes_22,mes_23,mes_24,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9
0,5001718,0,0,0,0,0,0,0,0,NaN,...,NaN,0,NaN,0,0,0,0,NaN,0,NaN
1,5001719,0,0,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
2,5001720,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,5001723,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,NaN,NaN
4,5001726,0,0,C,C,C,C,C,C,C,...,C,C,C,0,0,0,C,C,C,C


In [14]:
# Cria um DataFrame 'default_monthly' indicando se o valor mensal é maior ou igual a 2 (1 se verdadeiro, 0 se falso)
default_monthly = pagamento.iloc[:, 1:].apply(lambda col: pd.to_numeric(col, errors='coerce') >= 2).astype(int)

# Concatena a coluna 'ID' do DataFrame original com o DataFrame 'default_monthly'
default_monthly = pd.concat([pagamento['ID'], default_monthly], axis=1)

# Imprime o DataFrame resultante
print(default_monthly)

            ID  mes_0  mes_1  mes_10  mes_11  mes_12  mes_13  mes_14  mes_15  \
0      5001718      0      0       0       0       0       0       0       0   
1      5001719      0      0       0       0       0       0       0       0   
2      5001720      0      0       0       0       0       0       0       0   
3      5001723      0      0       0       0       0       0       0       0   
4      5001726      0      0       0       0       0       0       0       0   
...        ...    ...    ...     ...     ...     ...     ...     ...     ...   
20932  5150475      0      0       0       0       0       0       0       0   
20933  5150476      0      0       0       0       0       0       0       0   
20934  5150480      0      0       0       0       0       0       0       0   
20935  5150482      0      0       0       0       0       0       0       0   
20936  5150487      0      0       0       0       0       0       0       0   

       mes_16  ...  mes_22  mes_23  mes

In [15]:
# Adiciona a coluna 'STATUS' ao DataFrame 'propostas', indicando se há algum valor mensal maior ou igual a 2
propostas['STATUS'] = pagamento.iloc[:, 1:].apply(lambda col: (pd.to_numeric(col, errors='coerce') >= 2).any(), axis=1)

# Imprime as colunas 'ID' e 'STATUS' do DataFrame 'propostas'
print(propostas[['ID', 'STATUS']])

             ID STATUS
0       5008804  False
1       5008805  False
2       5008806  False
3       5008808  False
4       5008809  False
...         ...    ...
438552  6840104    NaN
438553  6840222    NaN
438554  6841878    NaN
438555  6842765    NaN
438556  6842885    NaN

[438557 rows x 2 columns]


In [16]:
# Filtra as linhas em 'propostas' onde o 'ID' está presente em 'pagamento'
clientes_tomadores = propostas[propostas['ID'].isin(pagamento['ID'])]

# Realiza um merge entre 'propostas' e 'default_monthly' com base na coluna 'ID' e preenche valores ausentes com 0
propostas = propostas.merge(default_monthly, on='ID', how='left').fillna(0)

# Imprime o DataFrame resultante
print(propostas)

             ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0       5008804           M            Y               Y             0   
1       5008805           M            Y               Y             0   
2       5008806           M            Y               Y             0   
3       5008808           F            N               Y             0   
4       5008809           F            N               Y             0   
...         ...         ...          ...             ...           ...   
438552  6840104           M            N               Y             0   
438553  6840222           F            N               N             0   
438554  6841878           F            N               N             0   
438555  6842765           F            N               Y             0   
438556  6842885           F            N               Y             0   

        AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0               427500.0      

In [17]:
# Calcula e imprime a contagem de valores na coluna 'STATUS' do DataFrame 'propostas'
contagem_default = propostas['STATUS'].value_counts()
print(contagem_default)

STATUS
False    438126
True        431
Name: count, dtype: int64
